In [ ]:
import sagemaker, boto3
# from sagemaker import get_execution_role

session = sagemaker.Session()
# this should be changed to get_execution_role() for book
role = "arn:aws:iam::941656036254:role/service-role/AmazonSageMaker-ExecutionRole-20210904T193230" 
account = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
# loging to Sagemaker ECR with Deep Learning Containers, so SageMaker can pull images in local mode
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
from sagemaker.local import LocalSession

sagemaker_local_session = LocalSession()
sagemaker_local_session.config = {'local': {'local_code': True}}

In [ ]:
import os
import subprocess

instance_type = "local"

try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
except:
    pass

print("Instance type = " + instance_type)

In [ ]:
from sagemaker.pytorch import PyTorch
import os

# Configure an MXNet Estimator (no training happens yet)
pytorch_estimator = PyTorch(
                        session=sagemaker_local_session,
                        entry_point=f'{os.getcwd()}/x_local_mode/train.py',
                        role=role,
                        instance_type=instance_type,
                        instance_count=1,
                        job_name="test",
                        framework_version="1.9.0",
                        py_version="py38",
                        hyperparameters={
                            "epochs": 1,
                            "batch-size": 16
                            }
                        )

pytorch_estimator.fit()


In [ ]:
endpoint = pytorch_estimator.deploy(initial_instance_count=1, instance_type=instance_type)

In [ ]:
# move this down on inference test

import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

trainset = torchvision.datasets.CIFAR10(
    root=os.getenv("SM_CHANNEL_TRAINING", "./"),
    train=True,
    download=True,
    transform=transform,
)



In [ ]:
import torch 

def _get_data_loader():
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    trainset = torchvision.datasets.CIFAR10(
        root=".",
        train=True,
        download=True,
        transform=transform,
    )
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=1, shuffle=True, num_workers=2
    )

    return trainloader


dataloader = _get_data_loader()

In [ ]:
item, label = next(iter(dataloader))

In [ ]:
import requests
import json 


payload = trainset[0][0].numpy().tobytes()
url = 'http://127.0.0.1:8080/invocations'
content_type = 'application/x-npy'
accept_type = "application/json"
headers = {'content-type': content_type, 'accept': accept_type}

response = requests.post(url, data=payload, headers=headers)
print(json.loads(response.content)[0])
